<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Using-bigger-data" data-toc-modified-id="Using-bigger-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Using bigger data</a></span><ul class="toc-item"><li><span><a href="#Training-data" data-toc-modified-id="Training-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Training data</a></span></li></ul></li><li><span><a href="#Test-Bktree" data-toc-modified-id="Test-Bktree-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Test Bktree</a></span></li></ul></div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
ls ..

__init__.py   data/         notebooks/    pyngramspell/


In [3]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [4]:
import data
from data import utils

train, test = utils.get_evaluation()

X_tr = [x['original'] for x in train]
y_tr = [x['corrected'] for x in train]
X_te = [x['original'] for x in test]
y_te = [x['corrected'] for x in test]

train_data = [' '.join(x) for x in y_tr]

In [6]:
import pyngramspell
speller = pyngramspell.PyNgramSpell()

In [7]:
speller.fit(train_data)

In [11]:
%%time
speller.transform('the maan went to the store')

CPU times: user 1.27 ms, sys: 16 µs, total: 1.29 ms
Wall time: 1.3 ms


['the', 'man', 'went', 'to', 'the', 'store']

In [13]:
%%time
speller.transform('red shirtt')

CPU times: user 230 µs, sys: 13 µs, total: 243 µs
Wall time: 245 µs


['red', 'shirtt']

In [14]:
%%time 
speller.transform('red shirtt')

CPU times: user 213 µs, sys: 10 µs, total: 223 µs
Wall time: 226 µs


['red', 'shirtt']

### Using bigger data

#### Training data


The dataset used to train the model comes from  https://jmcauley.ucsd.edu/data/amazon/

The `Video_Games_5.json` can be downloaded from 
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz

In [17]:
import pandas as pd
from os.path import expanduser
home = expanduser("~")

In [25]:
dataset_path = os.path.join(home,'Datasets/amazon_product_data/reviews_Video_Games_5.json')
df = pd.read_json(dataset_path, lines=True)

In [26]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2HD75EMZR8QLN,0700099867,123,"[8, 12]",Installing the game was a struggle (because of...,1,Pay to unlock content? I don't think so.,1341792000,"07 9, 2012"
1,A3UR8NLLY1ZHCX,0700099867,"Alejandro Henao ""Electronic Junky""","[0, 0]",If you like rally cars get this game you will ...,4,Good rally game,1372550400,"06 30, 2013"
2,A1INA0F5CWW3J4,0700099867,"Amazon Shopper ""Mr.Repsol""","[0, 0]",1st shipment received a book instead of the ga...,1,Wrong key,1403913600,"06 28, 2014"
3,A1DLMTOTHQ4AST,0700099867,ampgreen,"[7, 10]","I got this version instead of the PS3 version,...",3,"awesome game, if it did not crash frequently !!",1315958400,"09 14, 2011"
4,A361M14PU2GUEG,0700099867,"Angry Ryan ""Ryan A. Forrest""","[2, 2]",I had Dirt 2 on Xbox 360 and it was an okay ga...,4,DIRT 3,1308009600,"06 14, 2011"


In [27]:
train_data = list(df['reviewText'])

In [28]:
import pyngramspell
speller_bktree = pyngramspell.PyNgramSpell(min_freq=20,
                                           bktree=True)

speller_bktree.fit(train_data)

In [56]:
len(speller_bktree.vocabulary)

26484

In [57]:
speller_bktree.transform('xboj 360 colector edition')

['xbox', '360', 'collector', 'edition']

In [58]:
%%time
speller_bktree.transform('pssp blac edition')

CPU times: user 10.8 ms, sys: 5.9 ms, total: 16.7 ms
Wall time: 26.8 ms


['psp', 'black', 'edition']

In [59]:
%%time
speller_bktree.transform('new playstation game batmen')

CPU times: user 7.79 ms, sys: 2.31 ms, total: 10.1 ms
Wall time: 13.5 ms


['new', 'playstation', 'game', 'batman']

In [60]:
%%time
speller_bktree.get_candidates("pleystation", max_dist=3)

CPU times: user 22.9 ms, sys: 1.38 ms, total: 24.3 ms
Wall time: 23.8 ms


['levitation',
 'playstion',
 'playstaion',
 'playstation',
 'playstations',
 'playstation3',
 'playstation2']

In [48]:
speller_nobktree = pyngramspell.PyNgramSpell(min_freq=20,
                                           bktree=False)

speller_nobktree.fit(train_data)

In [50]:
%%time
speller_nobktree.transform('xboj 360 colector edition')

CPU times: user 135 ms, sys: 2.49 ms, total: 137 ms
Wall time: 138 ms


['xbox', '360', 'collector', 'edition']

In [55]:
%%time
speller_nobktree.transform('pssp blac edition')

CPU times: user 116 ms, sys: 1.97 ms, total: 118 ms
Wall time: 118 ms


['psp', 'black', 'edition']

In [54]:
%%time
speller_nobktree.transform('new playstation game batmen')

CPU times: user 70.8 ms, sys: 2.21 ms, total: 73 ms
Wall time: 73.4 ms


['new', 'playstation', 'game', 'batman']

In [53]:
%%time
speller_nobktree.get_candidates("pleystation", max_dist=3)

CPU times: user 76.3 ms, sys: 2.06 ms, total: 78.3 ms
Wall time: 78.2 ms


['playstations',
 'playstaion',
 'playstion',
 'levitation',
 'playstation3',
 'playstation2',
 'playstation']